In [1]:
import ants
import numpy as np
import os

In [2]:
_atlas_fiber_DTI_path = "tissue_reconstruction/data/FSL_HCP1065_tensor_1mm_space-HPC-AntsIndexSpace_SRI.nii.gz"
_atlas_t1_path = "tissue_reconstruction/data/sub-mni152_t1-inside-brain_space-sri.nii.gz"
_patient_path = "tissue_reconstruction/data/sample_patient.nii.gz"

_scalar_DTI_path = "tissue_reconstruction/data/FSL_HCP1065_FA_1mm_space-sri.nii.gz"



tensor_image = ants.image_read(_atlas_fiber_DTI_path)
atlas_t1 = ants.image_read(_atlas_t1_path)
patient = ants.image_read(_patient_path)

scalar_dti= ants.image_read(_scalar_DTI_path)

reg = ants.registration(patient, atlas_t1, "SyN")

transform = reg['fwdtransforms']



In [3]:
print(tensor_image)


ANTsImage (LPI)
	 Pixel Type : float (float32)
	 Components : 6
	 Dimensions : (240, 240, 155)
	 Spacing    : (1.0, 1.0, 1.0)
	 Origin     : (239.0, 0.0, 0.0)
	 Direction  : [-1.  0.  0.  0. -1.  0.  0.  0.  1.]



In [ ]:
from tqdm import tqdm

def reorient_tensor_image(patient, dti_image, warp):
       # Step 1: Split the 6-component DTI image
    dti_components = ants.split_channels(dti_image)
    
    # Step 2: Compute the Jacobian of the warp field
    jacobian_image = ants.create_jacobian_determinant_image(patient, warp, do_log=False)
    jacobian = jacobian_image.numpy()  # Convert to NumPy for easier manipulation
    
    # Initialize an empty array for the transformed components
    shape = dti_components[0].shape
    reoriented_dti = np.zeros((shape[0], shape[1], shape[2], 6))  # 6 components at each voxel
    
    # Step 3: Loop over each voxel and reorient the tensor
    for x in tqdm(range(shape[0])):
        for y in range(shape[1]):
            for z in range(shape[2]):
                # Get the tensor components at this voxel
                Dxx = dti_components[0][x, y, z]
                Dyy = dti_components[1][x, y, z]
                Dzz = dti_components[2][x, y, z]
                Dxy = dti_components[3][x, y, z]
                Dxz = dti_components[4][x, y, z]
                Dyz = dti_components[5][x, y, z]

                # Reconstruct the 3x3 tensor matrix
                T = np.array([[Dxx, Dxy, Dxz],
                              [Dxy, Dyy, Dyz],
                              [Dxz, Dyz, Dzz]])

                # Get the Jacobian matrix at this voxel (3x3 matrix)
                J = np.array(jacobian[x, y, z])

                # Reorient the tensor: T' = J * T * J^T
                reoriented_T = np.dot(J, np.dot(T, J.T))

                # Store the reoriented tensor components back into the image
                reoriented_dti[x, y, z, 0] = reoriented_T[0, 0]  # Dxx
                reoriented_dti[x, y, z, 1] = reoriented_T[1, 1]  # Dyy
                reoriented_dti[x, y, z, 2] = reoriented_T[2, 2]  # Dzz
                reoriented_dti[x, y, z, 3] = reoriented_T[0, 1]  # Dxy
                reoriented_dti[x, y, z, 4] = reoriented_T[0, 2]  # Dxz
                reoriented_dti[x, y, z, 5] = reoriented_T[1, 2]  # Dyz

    # Step 4: Create an ANTsImage for the reoriented tensor
    reoriented_dti_image = ants.from_numpy(reoriented_dti, origin=dti_image.origin, spacing=dti_image.spacing, direction=dti_image.direction)
    return reoriented_dti_image

In [7]:
def transform_tensor_img(fixed_image, dti_img, fwd_transform_paths):
    # preproccesing: Combine the transforms into one warp: TODO
    warp = fwd_transform_paths[0]
    
    # 1. Split into Six components
    dti_components = ants.split_channels(dti_img)
    #print(dti_components)

    # 2. Transform every component
    transformed_components = []
    for component in dti_components:
        transformed_component = ants.apply_transforms(fixed=fixed_image, 
                                                    moving=component, 
                                                    transformlist=fwd_transform_paths)
        transformed_components.append(transformed_component)
    transformed_dti_img = ants.merge_channels(transformed_components)

    print(transformed_dti_img)
    
    # 3. Reorient Image ?!
    transformed_dti_save_path = "out/transformed_dti.nii.gz"
    transformed_rotated_dti_save_path = "out/transformed_rotated_dti.nii.gz"

    ants.image_write(transformed_dti_img, transformed_dti_save_path)

    reorientTensorCmd = f"ReorientTensorImage 3 {transformed_dti_save_path} {transformed_rotated_dti_save_path} {warp}"
    os.system(reorientTensorCmd) 


def transform_tensor_img_my(fixed_image, dti_img, fwd_transform_paths):
    # preproccesing: Combine the transforms into one warp: TODO
    warp = fwd_transform_paths[0]

    # 1. Split into Six components
    dti_components = ants.split_channels(dti_img)

    #print(dti_components)

    # 2. Transform every component
    transformed_components = []
    for component in dti_components:
        transformed_component = ants.apply_transforms(fixed=fixed_image, 
                                                    moving=component, 
                                                    transformlist=fwd_transform_paths)
        transformed_components.append(transformed_component)
    transformed_dti_img = ants.merge_channels(transformed_components)

    #print(transformed_dti_img)
    
    # 3. Reorient Image ?!
    transf_reoriented_dti_img = reorient_tensor_image(patient, transformed_dti_img, warp)

    return transf_reoriented_dti_img



In [ ]:
transform_tensor_img(patient, tensor_image, transform)

transformed_tensor_img = ants.image_read("out/transformed_dti.nii.gz")
transformed_rotated_tensor_img = ants.image_read("out/transformed_rotated_dti.nii.gz")
transformed_rotated_tensor_img_my = transform_tensor_img_my(patient, tensor_image, transform)

print(transformed_tensor_img)
print(transformed_rotated_tensor_img)
print(transformed_rotated_tensor_img_my)


ANTsImage (RAI)
	 Pixel Type : float (float32)
	 Components : 6
	 Dimensions : (240, 240, 155)
	 Spacing    : (1.0, 1.0, 1.0)
	 Origin     : (-0.0, -239.0, 0.0)
	 Direction  : [1. 0. 0. 0. 1. 0. 0. 0. 1.]

FIELD: /tmp/tmpra72alxz1Warp.nii.gz
moving_image_filename: out/transformed_dti.nii.gz
output_image_filename: out/transformed_rotated_dti.nii.gz
[0/1]: FIELD: /tmp/tmpra72alxz1Warp.nii.gz
Warp transform
Iterating over image


In [ ]:
print("COMPARISON\n===========")

x, y, z = 100, 100, 100

transformed_tensor_arr = transformed_tensor_img.numpy()
transformed_rotated_tensor_arr = transformed_rotated_tensor_img.numpy()
transformed_rotated_tensor_arr_my = transformed_rotated_tensor_img_my.numpy()

print(transformed_tensor_arr[x, y, z])
print(transformed_rotated_tensor_arr[x, y, z])
print(transformed_rotated_tensor_arr_my[x, y, z])

print("Comparing my method with the Ants method")
print("MSE: ", np.mean(np.square(transformed_rotated_tensor_arr - transformed_rotated_tensor_arr_my)))
print("MAE: ", np.mean(np.abs(transformed_rotated_tensor_arr - transformed_rotated_tensor_arr_my)), "\n\n")

print("Comparing my method with the original tensor image")
print("MSE: ", np.mean(np.square(transformed_tensor_arr - transformed_rotated_tensor_arr_my)))
print("MAE: ", np.mean(np.abs(transformed_tensor_arr - transformed_rotated_tensor_arr_my)))

print("Comparing ANTS method with the original tensor image")
print("MSE: ", np.mean(np.square(transformed_tensor_arr - transformed_rotated_tensor_arr)))
print("MAE: ", np.mean(np.abs(transformed_tensor_arr - transformed_rotated_tensor_arr)))

COMPARISON
[ 7.9277722e-04  1.2565321e-05  7.2602270e-05  8.7643380e-04
 -3.5774083e-05  9.1422425e-04]
[ 7.8751025e-04  7.5883418e-06  7.0812632e-05  8.7899389e-04
 -3.2757089e-05  9.1693114e-04]
[ 7.9277722e-04  1.2565321e-05  7.2602270e-05  8.7643380e-04
 -3.5774083e-05  9.1422425e-04]
Comparing my method with the Ants method
MSE:  1.0849995e-11
MAE:  7.451862e-07 


Comparing my method with the original tensor image
MSE:  1.0849995e-11
MAE:  7.451862e-07
Comparing ANTS method with the original tensor image
MSE:  1.0849995e-11
MAE:  7.451862e-07
